In [7]:
import pandas as pd
import numpy as np

In [5]:
train = pd.read_csv('train.csv', index_col=0)

In [17]:
train['llm_idx'] = np.nan
train['llm_label'] = np.nan

In [12]:
answers = {
    '(A)': 'buildings', 
    '(B)': 'forest', 
    '(C)': 'glacier', 
    '(D)': 'mountain', 
    '(E)': 'sea', 
    '(F)': 'street'
}

In [59]:
# extract LLM labels from description
def extract_llm_label(description, is_first=False):
    idx = {}
    if '(A)' in description:
        idx['(A)'] = description.index('(A)')
    if '(B)' in description:
        idx['(B)'] = description.index('(B)')
    if '(C)' in description:
        idx['(C)'] = description.index('(C)')
    if '(D)' in description:
        idx['(D)'] = description.index('(D)')  
    if '(E)' in description:
        idx['(E)'] = description.index('(E)')
    if '(F)' in description:
        idx['(F)'] = description.index('(F)')
    if len(idx) == 2 and is_first:
        min_key = min(idx, key=idx.get)
        # Return a dictionary with the key and its minimum value
        return {min_key: idx[min_key]}

    return idx   

train['llm_idx'] = train['description'].apply(extract_llm_label)

In [23]:
# extract LLM label from description if len idx key is 1
def extract_llm_label_from_idx(idx):
    if len(idx) == 1:
        return answers[list(idx.keys())[0]]
    else:
        return np.nan

train['llm_label'] = train['llm_idx'].apply(extract_llm_label_from_idx)

# handle split string

In [34]:
error = train[train['llm_label'].isna()]

In [39]:
error['sub_description'] = error['description'].apply(lambda x: x.split('The main object in the image')[-1])

/tmp/ipykernel_339692/3223177211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['sub_description'] = error['description'].apply(lambda x: x.split('The main object in the image')[-1])


In [60]:
error['llm_idx'] = error['sub_description'].apply(extract_llm_label, is_first=True)

/tmp/ipykernel_339692/2769801696.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_idx'] = error['sub_description'].apply(extract_llm_label, is_first=True)


In [61]:
error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)

/tmp/ipykernel_339692/502944615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)


In [62]:
train['llm_label'] = train['llm_label'].fillna(error['llm_label'])

# get index by answer string

In [66]:
error = train[train['llm_label'].isna()]

In [104]:
# extract LLM labels from description
def extract_by_label(description, is_first=False):
    idx = {}
    if 'buildings' in description:
        idx['(A)'] = description.index('buildings')
    if 'forest' in description:
        idx['(B)'] = description.index('forest')
    if 'glacier' in description:
        idx['(C)'] = description.index('glacier')
    if 'mountain' in description:
        idx['(D)'] = description.index('mountain')  
    if 'sea' in description:
        idx['(E)'] = description.index('sea')
    if 'street' in description:
        idx['(F)'] = description.index('street')
    if is_first:
        min_key = min(idx, key=idx.get)
        # Return a dictionary with the key and its minimum value
        return {min_key: idx[min_key]}

    return idx   

error['llm_idx'] = error['description'].apply(extract_by_label, is_first=True)

/tmp/ipykernel_339692/330981100.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_idx'] = error['description'].apply(extract_by_label, is_first=True)


In [105]:
error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)

/tmp/ipykernel_339692/502944615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['llm_label'] = error['llm_idx'].apply(extract_llm_label_from_idx)


In [106]:
train['llm_label'] = train['llm_label'].fillna(error['llm_label'])

# FINISH

In [109]:
sum(train['label'] == train['llm_label']) / len(train) # 0.999

0.8870599971497791

In [ ]:
# save to csv
train.to_csv('trained.csv')